<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Other/Analysis/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# запросим предагреггированные данные
sql = """
SELECT
  card as client_id,
  Max((SELECT max(datetime) :: date FROM bonuscheques) - datetime :: date) as days_passed,
  count(*) as cnt,
  sum(summ_with_disc) as summ
FROM
  bonuscheques
GROUP BY
  card
"""
data = select(sql)

In [ ]:
# посмотрим, как выглядят данные
data.head()

,client_id,days_passed,cnt,summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577
1,2000200229544,234,2,1657
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111
3,2000200171775,236,2,1033
4,2000200229094,165,2,1235


Видим аномальный максимум в признаке cnt (Кто-то покупает почти каждый день?)

In [23]:
data = pd.read_csv('/content/bonuscheques.csv')

In [62]:
rfm_params = {'groupby_col': 'card', 'recentcy_col': 'datetime', 'monetary_col': 'summ_with_disc'}

In [ ]:
data

,datetime,shop,card,bonus_earned,bonus_spent,summ,summ_with_disc,doc_id,date
0,2021-07-13 12:56:09.000000,Аптека 2,2000200195023,51,0,3400,3400,15#2002741#65938#2_29,2021-07-13
1,2021-07-30 10:42:00.000000,Аптека 2,2000200193494,57,0,747,747,15#2002972#65955#2_5,2021-07-30
2,2021-10-11 12:55:23.000000,Аптека 2,2000200199106,92,253,3077,3077,15#2004060#66028#2_29,2021-10-11
3,2021-10-14 14:48:56.000000,Аптека 2,2000200168768,1,0,54,54,15#2004107#66031#2_57,2021-10-14
4,2021-10-20 11:09:39.000000,Аптека 2,2000200226314,101,0,1733,1733,15#2004192#66037#2_16,2021-10-20
...,...,...,...,...,...,...,...,...,...
38481,2022-06-09 20:45:32.000000,Аптека 11,2000200244974,32,0,1341,1341,15#18001790#66269#18_120,2022-06-09
38482,2022-06-09 20:55:32.000000,Аптека 11,2000200239246,38,0,1445,1445,15#18001790#66269#18_123,2022-06-09
38483,2022-06-09 20:59:41.000000,Аптека 11,2000200238867,57,0,1912,1912,15#18001790#66269#18_126,2022-06-09
38484,2022-06-09 21:07:59.000000,Аптека 11,2000200165811,2,0,278,278,15#18001790#66269#18_129,2022-06-09


In [69]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        groupby_col:str='card',
        recentcy_col:str='datetime',
        monetary_col:str='summ_with_disc',
        bins_quantiles:list|dict=[0.6]
    ):
        self.groupby_col = groupby_col
        self.recentcy_col = recentcy_col
        self.monetary_col = monetary_col
        self.bins_quantiles = bins_quantiles

        pass

    def fit(self, X):

        if not hasattr(X, "iloc"):
            raise ValueError(
                "CustomTransformer can only be applied to pandas dataframes in X argument"
            )

        if isinstance(self.bins_quantiles, list):
            self.bins_quantiles = sorted(self.bins_quantiles + [0, 1])

        return self

    def transform(self, X):
        if not hasattr(X, "iloc"):
            raise ValueError(
                "CustomTransformer can only be applied to pandas dataframes in X argument"
            )

        df = X[[self.groupby_col, self.recentcy_col, self.monetary_col]].copy()
        df.rename(columns={recentcy_col: 'date', monetary_col: 'summ'}, inplace=True)
        df['date'] = pd.to_datetime(df['date']).dt.date

        df = df.groupby(self.groupby_col).agg({'date': 'max', 'summ': ['count', 'sum']})
        df.columns = ['_'.join(col).strip() for col in df.columns.values]

        return df

    def inverse_transform(self):
        pass

In [57]:
pd.qcut(range(1, 11), [0, 0.25, 0.5, 0.75, 1], labels=False)

array([0, 0, 0, 1, 1, 2, 2, 3, 3, 3])

In [70]:
transformer = CustomTransformer()

In [71]:
transformer.get_params()

{'bins_quantiles': [0.6],
 'groupby_col': 'card',
 'monetary_col': 'summ_with_disc',
 'recentcy_col': 'datetime'}

In [73]:
transformer.fit_transform(data)

,date_max,summ_count,summ_sum
card,,,
001848d0-e56a-412e-be0a-423dfa0e564f,2022-05-14,7,4391
001cb0a0-e62e-47a9-995e-599bdc7f8750,2022-05-13,12,5887
00572591-c154-4b9f-83e4-3ce5b00bf9c8,2021-10-08,1,557
00625996-f49f-4863-8132-40d08e99b2da,2022-04-14,4,4755
00703805-f16d-4b83-b7ad-affb2736aa0d,2022-05-25,1,1555
...,...,...,...
ff9695c5-6e8c-4446-adc6-944b2731b8f7,2021-12-19,1,559
ffbf9d44-679e-412a-9927-ac6ce80c49a2,2021-08-27,1,539
ffd22196-662c-4266-b3be-024a42cad78b,2022-06-05,3,2052
